#### agent-101: 让AI自由组织工具回答问题

智能体在回答问题的时候，不一定只调用一个工具，而是可能需要综合调用多个工具。
比如：
- "帮我下载arxiv上与智能体相关的论文"包含搜索+下载两个动作，需要用到search和download两个工具
- "帮我全网搜索与量化投资相关的资料"就可能包含必应搜索、arxiv搜索、知网搜索等

这时候我们就需要指引工具选择智能体在选择工具的时候，输出一个工具列表而不是单一的工具。
比如：需要做多次搜索的时候，就需要输出类似如下内容：

"tools": ["get_bing_searched_results(keyword='大语言模型', max_results=5)", "get_bing_searched_results(keyword='Large Language Model', max_results=5)", "get_arxiv_papers(keyword='大语言模型', max_results=5, sort_by='relevance')", "search_cnki(keyword='大语言模型', num_pages=1)"]

选择三个工具，做了四次搜索

在这个智能体里边，更体现了大语言模型的智能，也就是规划能力。
我们给它提供足够多的工具，它能自由地选择工具，获取信息，回答我们的问题。

图中的例子里，我让我的智能体执行了全网搜索回答问题，以及搜索并下载论文到本地的功能。
- 全网搜索，调用了两个工具，搜索到了一万二个字的结果，让AI进行总结。
- 搜索并下载论文这个功能，有个比较复杂的地方就是，AI需要先搜索到论文的链接之后，才知道要下载的链接是什么，一开始的download肯定是不知道具体的参数要填什么的（也就是链接）。因此需要在搜索总结之后，再让tool agent选择工具，填入具体的参数才能执行下载。


到目前为止，基本上我们已经逐步搭建了比较完善的智能体系统：大模型+工具选择+计划+反思。
依然缺失的是记忆模块，我们下期再继续（挖坑ing）


##### to-do
- 搜索-下载-阅读
**TOP PRIORITY**
- 规划能力
- 记忆能力

**复杂RAG**
- 找到RAG评测数据集/知识库
- 将RAG做细
- 阅读开源框架（GraphRAG）
- Elastic Search
- faiss向量数据库

**企业微信小助手**
- 下载论文到本地的功能 √
- 聊天界面需要增加文件上传的功能 √
- 自动构建AT策略
- 自动挖掘因子
- 记住我们的喜好（参考mem0）
- 自主上网学习最新知识，保存到知识库，随时调取（参考GraphRAG构建知识图谱）
- 自我进化，每次都能输出最高质量的回答
- 完成论文中的实证部分
- 头脑风暴获取极具创意的想法
- 自主学习并构建全新的工具
- 多工具同时调用，汇总信息，给出答案
- 需要增加一个复杂问题解决流程，让智能体多次调用工具，综合多个工具的结果，最后给出答案。（使用"[END]"标记作为结束）
- 增加一个网页界面
- 随着聊天变长，随机遗忘不重要的信息。（可设计一个重要程度判断模型）
- 文件上传分析（读取文件、分析文件）
- 代码运行需要增加代码运行结果的展示
- 需要增加动态修改系统提示词的功能
- 增加标准工具箱，让新增工具的时候更方便
- files 多文件处理
- 知识图谱构建
- 路径规划


- 流式输出的时候增加多层思考过程，然后在最后修改/提供思路

**ERGENT**
- 工具规范调用（Mem0的mem0/llms/utils/tools.py文件）
    - 实现任意选择工具的功能（无工具、自选工具）
- 记忆功能
- 多工具转换为多智能体
- 

##### 最终实现
- 让智能体根据问题，自由调用工具实现任何问题
- 使用while - end 循环

- GPT-4o长输出：gpt-4o-64k-output-alpha


##### **多工具调用智能体**

In [3]:
from growing_agent_tools.llm_api import * 
from growing_agent_tools.json_tool import *
from growing_agent_tools.code_interpreter import * 

from growing_agent_tools.search_bing import *
from growing_agent_tools.search_arxiv import *
from growing_agent_tools.search_cnki import *
from growing_agent_tools.text2jupyter import *
from growing_agent_tools.pdf_reader import * 
from growing_agent_tools.download_arxiv_papers import *

class AllToolsAgent:
    
    def __init__(self, agent_name="AllTools"
                 , model="glm-4-0520"
                 , temperature=0.9):
        self.name = agent_name
        self.model = model
        self.temperature = temperature
        
        # 如果不存在files文件夹，则创建
        if not os.path.exists("files"):
            os.makedirs("files")
        # 获取files文件夹下的所有文件
        files = os.listdir("files")
        self.files = [f for f in files if f.endswith(".pdf")]
                
        # 工具选择系统
        self.tool_system_prompt = self.get_prompt("prompts/tools_system_prompt.txt")
        self.tool_system_prompt = self.tool_system_prompt.format(files=self.files)
        self.tool_conversations = [{"role": "system", "content": self.tool_system_prompt}]
        
        # 回答系统
        self.agent_system_prompt = '''你必须尽可能详细地回答我的问题。'''
        self.agent_conversations = [{"role": "system", "content": self.agent_system_prompt}]
        
    # 提示词读取
    def get_prompt(self, path):
        with open(path, 'r', encoding="utf-8") as file:
            prompt = file.read()
        return prompt
    
    # 依据聊天记录回答
    def get_answer_converse_yield(self):
        ans = ""
        for char in get_llm_answer_converse(self.agent_conversations, self.model, self.temperature):
            ans += char
            yield char

    # 选择工具
    def select_tools(self, question, tool_model="deepseek-chat"):
        # 注意事项
        notice = '''
# 注意
- 你的输出必须是一个json格式的字符串，这个json仅包含tool这个key，value是你选择的工具列表（每个工具包含函数名+参数）
- 你的json必须以```json开头，以```结尾
- 除非是你不熟悉的话题，否则不需要使用必应搜索工具，直接回答问题即可
- 除非是需要查找论文，否则搜索的时候不需要使用arxiv工具
- 你只需要输出json即可，不需要其他任何额外的输出
- 如果前面已经调用过提取PDF的工具，后续不需要再次调用，直接选择chat工具即可
'''
        # 选择工具
        self.tool_conversations.append({"role": "user", "content": "我现在要解决以下问题：\n" + question + "\n帮我选择合适的工具函数，以json格式发给我。" + notice})
        ans = ""
        for char in get_llm_answer_converse(self.tool_conversations, tool_model, self.temperature):
            ans += char
            print(char, end='', flush=True)
        self.tool_conversations.append({"role": "assistant", "content": ans})
        tools = get_json(ans)["tools"]
        return tools
    
    # 工作流
    def work_flow(self, question, tool_model="deepseek-chat"):
        total_info = ""
        # 选择工具
        tools = self.select_tools(question, tool_model)
        print(f"需要调用工具数量：{len(tools)} \n")
        for i,tool in enumerate(tools):
            print(f"第{i+1}个工具：{tool}")
            
        for tool in tools:   
            # 如果tool不是以download开头
            if not tool.startswith("download"):                    
                # arXiv搜索 下载arxiv论文 搜索CNKI 搜索必应 获取必应搜索结果 提取PDF文本
                if tool.startswith("get_arxiv_papers") or tool.startswith("search_cnki") or tool.startswith("get_bing_searched_results") or tool.startswith("extract_clean_text_from_pdf"):
                    yield "正在执行：\n" + tool + "\n"
                    result, _ = run_code_v2(tool, globals=globals())
                    print(f"工具运行结果如下：{str(result)[:1000]}\n")
                    total_info += str(result) + "\n"
                
                # 深度数据分析
                # code interpreter
                elif tool.startswith("run_code_v2"):
                    yield "正在运行代码..."
                    # 写代码
                    self.agent_conversations.append({"role": "user", "content": question})
                    ans = ""
                    for char in auto_code_running_modify(question, self.model):
                        yield char
                        ans += char
                    total_info += ans + "\n=====================\n"   
                                            
                prompt = '''
# 你的任务
根据提供的信息，回答我的问题

# 你帮我找到了以下信息：
"""
{total_info}
"""

# 我的问题
"""
{question}
"""

# 你的工作流
1. 你必须首先详细回答我上述的问题，不需要担心token限制，并且你必须以类似如下的格式回答我的问题：
"根据我找到的信息，我认为......"
2. 然后在最后简要列举你搜索到的所有信息+链接，作为参考

# 参考输出
根据我找到的信息...

# 相关链接
- [链接1](https://...)
...

# 注意
- 相关链接只需要在第二步展示即可，第一步不需要提供链接
- 你拥有下载文件的工具，如果需要下载文件，则告诉我请稍后即可
现在，请回答我的问题：
{question}
'''
                prompt = prompt.format(total_info=total_info, question=question)
                self.tool_conversations.append({"role": "user", "content": prompt})    
                self.agent_conversations.append({"role": "user", "content": prompt})   
                print("\n" + "="*20 + f"工具调用结束, 获取信息字数：{len(total_info)}，开始回答" + "="*20 + "\n") 
                ans = ""
                for char in self.get_answer_converse_yield():
                    yield char
                    ans += char
                self.tool_conversations.append({"role": "assistant", "content": ans[:1000]})
                self.agent_conversations.append({"role": "assistant", "content": ans})      
            else:
                # 下载文件
                question = "现在，请根据上述搜索到的链接，编写函数以及参数，给出json格式的工具"
                tools = self.select_tools(question, tool_model)
                yield "\n正在执行：\n" + tools
                [0] + "\n"
                result, _ = run_code_v2(tools[0], globals=globals())
                print(f"工具运行结果如下：{str(result)}\n")
                yield "下载成功"

question = "请帮我全网搜索关于智能体的最新信息，并详细分点汇总。"
question = "帮我搜索3篇最新的与智能体相关的论文，详细介绍，然后下载到本地。"
question = "什么是大语言模型？"
print(f"问题：{question}\n")
model = "glm-4-0520"
model = "gpt-4o-2024-08-06"

agent = AllToolsAgent(model=model)
for char in agent.work_flow(question):
    print(char, end='', flush=True)

问题：什么是大语言模型？



2024-08-08 21:15:08,128 - INFO - HTTP Request: POST https://api.deepseek.com/v1/chat/completions "HTTP/1.1 200 OK"


```json
{
    "tools": ["chat"]
}
```需要调用工具数量：1 

第1个工具：chat

====================工具调用结束, 获取信息字数：0，开始回答====================



2024-08-08 21:15:10,517 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


根据我找到的信息，我认为大语言模型（Large Language Model, LLM）是一种基于机器学习的人工智能模型，专门用于处理和生成自然语言文本。这些模型是通过在大量的文本数据上进行训练来掌握语言的结构、语法、语义和背景知识。其主要特点包括：

1. **规模庞大**：大语言模型通常包含非常多的参数，比如数十亿到上千亿个参数，这使得它们有能力捕获更加复杂的语言模式。

2. **深度学习架构**：它们主要基于深度学习架构，例如Transformers，这种架构使模型能有效地处理长距离的依赖关系和上下文信息。

3. **预训练与微调**：大语言模型通常先在广泛的语料库上进行无监督的预训练，学习到一般的语言特征，然后通过微调（fine-tuning）在特定任务（如翻译、文本生成、情感分析等）上进行调整。

4. **生成与理解能力**：这些模型能够生成符合语法且有逻辑的文本，回答问题，翻译语言，以及执行其他自然语言处理任务。

5. **多领域应用**：大语言模型可以应用于多个领域，包括但不限于客服自动化、内容创作、翻译服务、代码生成、教育等。

然而，大语言模型也面临一些挑战，比如在生成内容中的偏见与不准确性，计算资源消耗大，以及在某些任务上的不确定性或不稳定性。

# 相关链接
这里没有提供具体的链接，如果需要进一步深入的信息或者引用，可以请求具体的文献或者在线资源。

In [ ]:
question = "用几句话分别总结一下这十篇论文的主要内容"

for char in agent.work_flow(question):
    print(char, end='', flush=True)

In [5]:
print(agent.agent_conversations[-2:-1][0]["content"])


# 你的任务
根据提供的信息，回答我的问题

# 你帮我找到了以下信息：
"""
关于智能体的搜索结果如下：
Link: https://baike.baidu.com/item/%E6%99%BA%E8%83%BD%E4%BD%93/9446647
智能体 目录 简介 概念的提出 定义 特性 区别内容 应用架构 社会应用 工作环境 环境定义 环境特性 组成内容 观察和感知 记忆和检索 推理和规划 行动和执行 ©2024 Baidu使用百度前必读|百科协议|隐私政策|百度百科合作平台|京ICP证030173号 京公网安备11000002000001号
Link: https://new.qq.com/rain/a/20231013A03U0A00
大语言模型时代的智能体 (I)：什么是智能体？
Link: https://www.gnomic.cn/home ==================================================
Link: https://www.wolai.com/cUCxuiNAQYJcpaYxCvR1QU ==================================================
Link: https://blog.csdn.net/2301_81940605/article/details/136870012
什么是智能体(agent) 智能体（Agent）是人工智能领域中的一个核心概念。在最基本的层面上，智能体可以被定义为一个实体，它能够在其所处的环境中自主地感知信息，并根据这些信息做出决策，以实现特定的目标或任务。智能体的关键特性包括自主性、感知能力和决策能力。 智能体的目标可以是简单的，如维持系统稳定，也可以是复杂的，如在多智能体系统中进行协调合作。智能体的设计和实现是为了解决特定的问题，它们可以在多种环境中运作，包括物理世界和虚拟世界。 在接下来，我们将详细探讨智能体的组成，以及它们是如何在不同的环境和应用中发挥作用的。 智能体的组成 智能体的组成是其功能实现的基础。一个典型的智能体由以下几个主要部分组成： 智能体的这些组成部分相互作用，使其能够在环境中自主地运作。感知器提供输入数据，决策制定机制处理这些数据并决定行动，执行器执行决策结果，而知识库和学习机制则不断更新和优化

In [ ]:
from growing_agent_tools.search_cnki import *
keyword = "智能体"
num_pages = 1
use_proxy = False
proxy = None

results_matrix = search_cnki(keyword, num_pages, use_proxy, proxy)

In [ ]:
results_matrix